# Instalando a função e Importando Bibliotecas
Vamos usar agora nessa parte uma função para retirar o ruído dos áudios do nosso dataset, para posteriormente calcularmos o nosso SNRs.

In [ ]:
!pip install -U denoiser

In [ ]:
from IPython import display as disp
import torch
import torchaudio
from denoiser import pretrained
from denoiser.dsp import convert_audio

# Retirando os ruídos do áudio
Aqui vamos retirar os ruídos dos nossos áudios, no caso vamos usar só a pasta dos nossos áudios originais, não vamos precisar do CSV nesse momento, vale ressaltar que vamos salvar os novos áudios em uma nova pasta.

In [ ]:
import torchaudio
import os
import glob
import torch  # Garanta que torch esteja importado para usar torch.no_grad()

# Diretório onde os arquivos de áudio originais estão localizados
source_dir = '/content/audios' # Mude para a sua pasta com seus áudios originais

# Diretório de destino para salvar os arquivos denoised
dest_dir = '/content/drive/MyDrive/DataSet/denoised' # Altere para onde você quiser salvar os áudios sem ruido.
os.makedirs(dest_dir, exist_ok=True)  # Cria o diretório de destino se ele não existir
contagem_processados = 0
# Lista todos os arquivos de áudio na pasta 'audios'
audio_files = glob.glob(os.path.join(source_dir, '*'))

# Assegure-se de que o modelo está sendo carregado na GPU
# Usamos esse modelo para fazer essa remoção de ruído, você pode mudar caso queira.
model = pretrained.dns64().to('cuda')

for audio_path in audio_files:
    # Constrói o caminho de destino com o mesmo nome do arquivo original
    base_name = os.path.basename(audio_path)
    denoised_file_path = os.path.join(dest_dir, base_name)

    # Verifica se o arquivo já existe no diretório de destino
    if os.path.exists(denoised_file_path):
        print(f"{contagem_processados} - Arquivo {base_name} já processado e disponível em: {denoised_file_path}")
        contagem_processados += 1
        continue  # Pula para o próximo arquivo se este já foi processado

    wav, sr = torchaudio.load(audio_path)
    # Converte e move o tensor para a GPU
    wav = convert_audio(wav.to('cuda'), sr, model.sample_rate, model.chin)
    with torch.no_grad():
        denoised = model(wav[None])[0]

    # Salvando o áudio denoised (certifique-se de mover o tensor de volta para a CPU antes de salvar)
    torchaudio.save(denoised_file_path, denoised.cpu(), sample_rate=model.sample_rate)

    print(f"{contagem_processados} - Arquivo denoised salvo em: {denoised_file_path}")
    contagem_processados += 1


In [ ]:
# Buscamos o total de arquivos para ver se corresponde com a mesma quantidade de áudios com ruído
num_files = len([name for name in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, name))])

print(f"Total de arquivos no diretório '{source_dir}': {num_files}")

In [ ]:
# Buscamos o total de arquivos para ver se corresponde com a mesma quantidade de áudios com ruído e sem ruído
num_files = len([name for name in os.listdir(dest_dir) if os.path.isfile(os.path.join(dest_dir, name))])

print(f"Total de arquivos no diretório '{dest_dir}': {num_files}")